## (b) Stochastic Optimization: Genetic Algorithm on TSP

In [2]:
from math import cos
from math import sin
from math import pi
from random import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

In [3]:
N = 8
citynames = ['1', '2', '3', '4', '5', '6', '7', '8']
citydistances = np.array([[0, 0.6347, 0.3231, 0.2510, 0.2727, 0.4968, 0.5144, 0.2105],
 [0.6347, 0, 0.9375, 0.5755, 0.6472, 0.2507, 0.2808, 0.7916],
 [0.3231, 0.9375, 0, 0.4092, 0.3636, 0.8195, 0.7615, 0.1527],
 [0.2510, 0.5755, 0.4092, 0, 0.0730, 0.5427, 0.3577, 0.2577],
 [0.2727, 0.6472, 0.3636, 0.0730, 0, 0.6136, 0.4204, 0.2187],
 [0.4968, 0.2507, 0.8195, 0.5427, 0.6136, 0, 0.4048, 0.6918],
 [0.5144, 0.2808, 0.7615, 0.3577, 0.4204, 0.4048, 0, 0.6088],
 [0.2105, 0.7916, 0.1527, 0.2577, 0.2187, 0.6918, 0.6088, 0]]) # a matrix containing city to city distance

In [4]:
citydistances

array([[0.    , 0.6347, 0.3231, 0.251 , 0.2727, 0.4968, 0.5144, 0.2105],
       [0.6347, 0.    , 0.9375, 0.5755, 0.6472, 0.2507, 0.2808, 0.7916],
       [0.3231, 0.9375, 0.    , 0.4092, 0.3636, 0.8195, 0.7615, 0.1527],
       [0.251 , 0.5755, 0.4092, 0.    , 0.073 , 0.5427, 0.3577, 0.2577],
       [0.2727, 0.6472, 0.3636, 0.073 , 0.    , 0.6136, 0.4204, 0.2187],
       [0.4968, 0.2507, 0.8195, 0.5427, 0.6136, 0.    , 0.4048, 0.6918],
       [0.5144, 0.2808, 0.7615, 0.3577, 0.4204, 0.4048, 0.    , 0.6088],
       [0.2105, 0.7916, 0.1527, 0.2577, 0.2187, 0.6918, 0.6088, 0.    ]])

In [16]:
# Genetic Search
ngen = 1000
ngpool = N

gpool = np.zeros((ngpool,N)).astype('int')
 
for i in range(ngpool):
  # We ensure that the first element remains 0
#  if i == 0:
#    gpool[i] = range(N)
#  else:
    gpool[i,1:] = (np.random.permutation(N-1) + 1)

    costmin = N
    tourmin = np.zeros(N)
    cost = np.zeros(ngpool)
for i in range(ngen):
    # step 1. evaluate the fitness of current chromosomes
    # for tsp problem it is the trip length. the smaller the better
    for j in range(ngpool):
        id_shift = gpool[j,-1:].tolist() + gpool[j,:-1].tolist()
        cost[j] = np.sum(np.diag(citydistances[gpool[j,:].tolist(),id_shift]))

    # Record the current best solution
    costmin = np.amin(cost)
    idx = np.argmin(cost)
    tourmin = gpool[idx,:]

    #print(cost)
    #print(costmin)
    # step 2. cross breeding and mutation
    # since each chromosome is an integer vector, cross breeding
    # can not be done simply by combining binary vectors. 
    # Instead, we let the off-spring to keep the common genes of
    # the parents, and randomly shuffle genes when parents disagree
    # for simplicity, we only cross-breed the two best solutions. 

    ridx = sorted(range(len(cost)), key=lambda k: cost[k])


    # parents
    mother = gpool[ridx[0],:]
    father = gpool[ridx[1],:]
    # find index of genes that are the same in father and mother
    sameidx = father == mother
    diffidx = np.nonzero(1 - sameidx)[0]


    if len(diffidx <= 4): # father and mother too close!
      # do mutation
        boy = [0] + (np.random.permutation(N-1) + 1).tolist()
        girl = [0] + (np.random.permutation(N-1) + 1).tolist()
    else:
        boy = father * sameidx
        boy[diffidx] = father[diffidx[np.random.permutation(len(diffidx))]]
        girl = mother * sameidx
        girl[diffidx] = mother[diffidx[np.random.permutation(len(diffidx))]]
  
    #Replace the worst two
    gpool[ridx[-2]] = boy
    gpool[ridx[-1]] = girl

In [17]:
print("cost function evaluation: " + str(ngen*2 + ngpool) + " times!")
print("Minimum trip length = " + str(costmin))
print("optimum tour = " + str(tourmin))

cost function evaluation: 2008 times!
Minimum trip length = 2.1535
optimum tour = [0 2 7 4 3 6 1 5]


In [18]:
citylist = [];
for i in range(N):
    citylist.append(citynames[tourmin[i]])

In [19]:
print("The corresponding cities",citylist)

The corresponding cities ['1', '3', '8', '5', '4', '7', '2', '6']
